# EDA for Walmart Weekly Sale

In [59]:
import pandas as pd
import numpy as np
from numpy import *

import gc
import time
import warnings
import os

In [60]:
data_dir = 'data'
train_file_name = 'train.csv'
features_file_name = 'features.csv'
store_file_name = 'stores.csv'


In [61]:
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [78]:
train_data.Store.unique

<bound method Series.unique of 0          1
1          1
2          1
3          1
4          1
5          1
6          1
7          1
8          1
9          1
10         1
11         1
12         1
13         1
14         1
15         1
16         1
17         1
18         1
19         1
20         1
21         1
22         1
23         1
24         1
25         1
26         1
27         1
28         1
29         1
          ..
421540    45
421541    45
421542    45
421543    45
421544    45
421545    45
421546    45
421547    45
421548    45
421549    45
421550    45
421551    45
421552    45
421553    45
421554    45
421555    45
421556    45
421557    45
421558    45
421559    45
421560    45
421561    45
421562    45
421563    45
421564    45
421565    45
421566    45
421567    45
421568    45
421569    45
Name: Store, Length: 421570, dtype: int64>

In [62]:
# train_data.info()
# train_data['Weekly_Sales'].describe()

In [63]:
train_df = train_data.copy()
# train_df.groupby('Store').groups
# # train_df['Store'].value_counts().sort_values(ascending=True, by='Store')
# train_df['Store'].value_counts().sort_values(ascending=True)

In [64]:
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [65]:
train_df = train_df.rename(columns={'Date':'date', 'Store':'item_id'})
train_df.head()

,item_id,Dept,date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [66]:
store_1_2010_02_05_sale = train_df.query('date == "2010-02-05" &  item_id == 1')['Weekly_Sales'].sum()
assert (store_1_2010_02_05_sale == 1643690.90)

# train_df['Date'] == '2010-02-05' and train_df['Store'] == 1

In [67]:
temp_df = train_df.groupby(['item_id','date'])['Weekly_Sales'].sum()
# df2 = pd.DataFrame(temp_df, columns=['Store','Date','Weekly_Sales'])
stores_sales = pd.DataFrame(temp_df)
stores_sales.head()



Weekly_Sales
item_id date                    
1       2010-02-05    1643690.90
        2010-02-12    1641957.44
        2010-02-19    1611968.17
        2010-02-26    1409727.59
        2010-03-05    1554806.68

In [68]:
# df2.set_index('Date', inplace=True)
stores_sales.reset_index(inplace=True)
# print(stores_sales.groupby('Store').agg({'Date': ['min','max']}))
stores_sales = stores_sales.set_index('date')
stores_sales.head()



,item_id,Weekly_Sales
date,,
2010-02-05,1,1643690.90
2010-02-12,1,1641957.44
2010-02-19,1,1611968.17
2010-02-26,1,1409727.59
2010-03-05,1,1554806.68


In [69]:

stores_sales.item_id = stores_sales.item_id.astype(str)
stores_sales.dtypes
# df['date'] = pd.to_datetime(df["date"].dt.strftime('%Y-%m'))

# df1.index = pd.to_datetime(df1.index, format = '%m/%d/%Y')
# df1.index = df1.index.strftime('%Y-%m-%d')

# >>> a = datetime.strptime("10/12/2013", "%m/%d/%Y")
stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )
# stores_sales.index = stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss")
stores_sales.index = stores_sales.index.strftime('%Y-%m-%d %H:%M:%S')

#stores_sales.index = pd.to_datetime(stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss"))
cols = ['Weekly_Sales', 'item_id']
stores_sales = stores_sales[cols]

In [70]:
stores_sales.head()

,Weekly_Sales,item_id
2010-02-05 00:00:00,1643690.90,1
2010-02-12 00:00:00,1641957.44,1
2010-02-19 00:00:00,1611968.17,1
2010-02-26 00:00:00,1409727.59,1
2010-03-05 00:00:00,1554806.68,1


In [71]:

target_stores_sales = stores_sales[stores_sales.index < '2011-10-01']
validation_stores_sales = stores_sales[stores_sales.index >= '2011-10-01']

# target_stores_sales = target_stores_sales.['Date' '2011-12-31']
# target_stores_sales

In [77]:
target_stores_sales.tail()

,Weekly_Sales,item_id
2011-09-02 00:00:00,726482.39,45
2011-09-09 00:00:00,746129.56,45
2011-09-16 00:00:00,711367.56,45
2011-09-23 00:00:00,714106.42,45
2011-09-30 00:00:00,698986.34,45


In [73]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [74]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

In [75]:
region = 'us-east-2'

In [76]:
%store region
%store target_stores_sales
%store validation_stores_sales
%store target_time_series_filename
%store target_time_series_path
%store validation_time_series_filename
%store validation_time_series_path

Stored 'region' (str)
Stored 'target_stores_sales' (DataFrame)
Stored 'validation_stores_sales' (DataFrame)
Stored 'target_time_series_filename' (str)
Stored 'target_time_series_path' (str)
Stored 'validation_time_series_filename' (str)
Stored 'validation_time_series_path' (str)


In [20]:
validation_stores_sales.tail()

,Weekly_Sales,item_id
2012-09-28 00:00:00,713173.95,45
2012-10-05 00:00:00,733455.07,45
2012-10-12 00:00:00,734464.36,45
2012-10-19 00:00:00,718125.53,45
2012-10-26 00:00:00,760281.43,45


# EDA 2

In [101]:
# stores_sales.rename(columns={'Date':'date_time', 'Store':'item_ID'})
# stores_sales = stores_sales.set_index('Date')
# stores_sales.index

stores_1_sales = stores_sales[stores_sales['item_id']== "1"]
stores_1_sales[stores_1_sales.index >= '2011-12-20']

,Weekly_Sales,item_id
2011-12-23 00:00:00,2270188.99,1
2011-12-30 00:00:00,1497462.72,1
2012-01-06 00:00:00,1550369.92,1
2012-01-13 00:00:00,1459601.17,1
2012-01-20 00:00:00,1394393.84,1
2012-01-27 00:00:00,1319325.59,1
2012-02-03 00:00:00,1636339.65,1
2012-02-10 00:00:00,1802477.43,1
2012-02-17 00:00:00,1819870.00,1
2012-02-24 00:00:00,1539387.83,1


In [19]:

total_weeks = stores_sales.groupby('Store').count().sum() 
assert (45 * 143 == int(total_weeks) )
print(stores_sales.index.min())
print(stores_sales.index.max())

2010-02-05 00:00:00
2012-10-26 00:00:00


In [20]:
print(total_weeks)

Weekly_Sales    6435
dtype: int64


In [ ]:
stores_sales[stores_sales['Store']==1]['Weekly_Sales'].plot(title='Store 1', figsize=(10,5))

In [ ]:
# Guide: Multiple subplots
# https://jakevdp.github.io/PythonDataScienceHandbook/04.08-multiple-subplots.html

In [ ]:
import matplotlib.pyplot as plt
x = stores_1_sales['Date']
y = stores_1_sales['Weekly_Sales']

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(1, 6):
    ax = fig.add_subplot(2, 3, i)
    ax.plot(x,y)
    # ax.figsize(3,2)

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(1, 7):
    ax = fig.add_subplot(2, 3, i)
    ax.text(0.5, 0.5, str((2, 3, i)),
           fontsize=18, ha='center')

In [ ]:
def show_two_chart_horizontal(title, df1, sub_title1, df2, sub_title2):
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle(title)

    fig.set_figwidth(16)
    fig.set_figheight(4)

    ax1.title.set_text(sub_title1)
    ax2.title.set_text(sub_title2)
    df1.plot(ax=ax1)
    df2.plot(ax=ax2)

In [ ]:
features_data = pd.read_csv(os.path.join(data_dir,features_file_name))
features_data.head()

In [ ]:
store_data = pd.read_csv(os.path.join(data_dir,store_file_name))
store_data.head()